<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Методы релаксации

### Modules  – Partial Differential Equations
<section class="post-meta">
By Jonas Tjemsland, Eilif Sommer Øyre and Jon Andreas Støvneng
</section>
Last edited: February 7th 2018 
___

Методы релаксации включают в себя итеративное решение системы уравнений, возникающей из конечно-разностной аппроксимации, до тех пор, пока не будет найдено решение [[1]](#rsc1). Мы введем *метод Якоби*, *метод Гаусса-Зейделя* и *последовательную сверхрелаксацию* (SOR). Эти методы могут быть использованы для решения эллиптического уравнения

$$\mathcal L \psi = \rho,$$

где $\mathcal L$ является эллиптическим оператором, а $\rho$ - источником.

Методы просты для понимания на конкретном примере. Мы будем решать двумерное уравнение Пуассона,

\begin{equation}
\nabla^2 \psi(x, t) = \frac{\partial^2 \psi(x, y)}{\partial x^2} + \frac{\partial^2 \psi(x, y)}{\partial y^2} = \rho(x, y),
\label{eq:poisson} \quad (1)
\end{equation}

для квадратной области $\Omega = (0,L)\times (0, L)$, $L\in \mathbb R_+$. Без потери общности пусть $L=1$.

## Уравнение Пуассона в физике

Уравнения Пуассона и Лапласа $(\rho = 0)$ широко используются в физике. В электростатике электрический потенциал описывается $\nabla^2 V\left(\mathbf r\right) = -\rho \left(\mathbf r\right)/\epsilon_0,$ где $V$ - электрический потенциал, а $\epsilon$ - электрическая диэлектрическая проницаемость (это дифференциальная форма закона Гаусса для электрического потенциала. См., например, [[2]](#rsc2)). Стационарное состояние уравнения диффузии имеет вид $u_t = \nabla^2 u - \rho = 0$. Для устойчивого несжимаемого потока в двух измерениях функция потока удовлетворяет $\nabla^2 \psi = -\zeta$, где $\zeta=\nabla\times \mathbf V$ - завихренность, а поле скорости задается $\mathbf V = (u, v) = (\psi_y, -\psi_x)$ (См., например: [[3]](#rsc3)).

Решение является уникальным, когда граничные условия для всей системы четко определены. Это более подробно обсуждается в [приложении](#appendix).

## Метод Якоби

Рассмотрим уравнение

\begin{equation}
\frac{\partial \psi(x, y, t)}{\partial t}=\frac{\partial^2 \psi(x, y, t)}{\partial x^2} + \frac{\partial^2 \psi(x, y, t)}{\partial y^2} - \rho(x, y),
\label{eq:diffusion}\quad (2)
\end{equation}

чье устойчивое состояние является уравнением Пуассона (1). Пусть $x$, $y$ дискретизируются в некоторую сетку,

$$x_i = x_0 + i\Delta x, \: (\:i=0, 1, ...,N)$$

$$y_i = y_0 + j\Delta y, \: (\:j=0, 1, ..., N)$$ 
и
$$t_n = n\,\delta t, \: (\:n = 0, 1, ..., n).$$

Поскольку мы рассматриваем квадратную область $\Omega$, у нас есть $x_0 = y_0 = 0$ и $x_n = y_m = 1$. Чтобы упростить обозначение, мы вводим $u(x_i, y_j, t_n)\equiv u_{i, j}^n$ and $\rho(x_i, y_j) \equiv \rho_{i, j}$.

Если мы используем центральную разность для пространственных производных и правую - для дифференцирования по времени (FTCS) для уравнения (2), мы получим

$$\frac{\psi^{n+1}_{i, j} - \psi^n_{i, j}}{\Delta t} = \frac{\psi^n_{i+1, j}-2\psi^n_{i, j}+\psi^n_{i-1, j}}{(\Delta x)^2} + \frac{\psi^n_{i, j+1}-2\psi^n_{i, j}+\psi^n_{i, j-1}}{(\Delta y)^2} - \rho_{i, j}.$$

Для удобства определим $\Delta \equiv \Delta x=\Delta y$:

\begin{equation}
\psi^{n+1}_{i, j} = \psi^n_{i, j} + \frac{\Delta t}{\Delta^2} \left(\psi_{i+1, j}^n + \psi_{i-1, j}^n + \psi_{i, j+1}^n + \psi_{i, j-1}^n - 4\psi_{i, j}^n\right) - \Delta t \,\rho_{i, j}.
\label{eq:poisson_differenced}\quad (3)
\end{equation}

Можно показать, что разность FTCS для уравнения Пуассона в двух измерениях стабильна только в том случае, если $\Delta t/\Delta^2 \leq 1/4$ (см. [Приложение](#appendix)). Если мы установим $\Delta t/\Delta^2 = 1/4$, мы получим

\begin{equation}
\psi^{n+1}_{i, j} = \frac{1}{4} \left(\psi_{i+1, j}^n + \psi_{i-1, j}^n + \psi_{i, j+1}^n + \psi_{i, j-1}^n\right) - \frac{\Delta ^2}{4} \,\rho_{i, j}.
\label{eq:jacobi_method}\quad (4)
\end{equation}

> Таким образом, алгоритм состоит в выборе значения $(x_i, y_j)$ в качестве среднего значения соседних точек плюс некоторая неоднородность $\rho$.

Мы можем использовать любую первоначальную догадку. Итерация повторяется до тех пор, пока не будет удовлетворен критерий сходимости. Мы будем использовать общее относительное изменение между двумя итерациями в качестве условия остановки. Конечным решением является стационарное решение уравнения (2), которое является уравнением Пуассона (1).

#### Пример
Рассмотрим квадратный домен без каких-либо источников. Уравнение Пуассона сводится к уравнению Лапласа $(\rho = 0)$ внутри области. Граничные условия указаны на рисунке ниже. Мы выбираем $L=1$, $a=0$ и $b = 1$ (с необходимыми размерами) без потери общности.

![Example - Square setup](images/relaxation_square_example.png)
**Figure 1:** *Значение $\psi$ вдоль сплошных линий равно $a$ и $b$. $\psi$ равномерно варьируются между $a$ и $b$ вдоль пунктирных линий.*

Данная постановка может быть применима, например, к моделированию проводника (верхняя левая часть, $a$) и проводника с равномерно распределенным зарядом (нижняя правая часть, $b$). Электрическое поле, в свою очередь, задается $\mathbf E = \nabla \psi$. Установка также может моделировать поток несжимаемой и невращающейся жидкости через квадратную полость с равномерным притоком и оттоком. Приток и отток - это точечные линии, соответственно справа и снизу. Поле скорости задается $\mathbf V = (\partial_y \psi, - \partial_x \psi)$. Мы отсылаем вас к нашим отдельным блокнотам по потоковым функциям и электромагнетизму для получения дополнительной информации.

Для решения уравнения Лапласа для этой установки мы используем метод Якоби. Мы определяем следующие функции.

In [ ]:
import numpy as np

def bc(n=32):
    """ Представляет граничные условия psi, как показано на рисунке 1
        в 2D-матрице.
    
    Параметры:
        n :     int. Количество шагов в x - и y-
                направлениях
    Возвращается:
    int array, size (n, n). Граничные условия.
    """
    
    psi = np.zeros((n, n))
    width = int(n/3)

    # Граничные условия
    psi[0, 2*width:] = 1  # Нижняя левая часть b
    psi[:width, -1] = 1   # верхняя правая часть b
    psi[0, width:2*width] = np.linspace(0, 1, width)  # Приток
    psi[width:2*width, -1] = np.linspace(1, 0, width) # Отток
    # Остальные уже равны нулю
    
    return psi

def jacobi(n=32, maxit=1000, tol=1e-10):
    """ Эта функция использует метод Якоби для решения задачи Пуассона
    для квадратной области и граничных условий, как
    показано на рисунке.
    
    Параметры:
        n :     int. Количество шагов в направлениях x и y
        maxit : int. Максимальное количество итераций.
        tol :   float. Допустимая погрешность.
                Если общее относительное изменение между двумя
                итерациями меньше, чем "tol", мы говорим, что метод сошелся.
    Возвращается:
    float array, size(n, n). Найденое решение. 
    """
    
    psi = bc(n)
    psi_temp = psi.copy()
    itnum = 1             # Счетчик итераций
    change = tol + 1      # Общее относительное квадратичное изменение между каждой итерацией
    
    for itnum in range(maxit):
        # Пусть каждая точка является средним значением ее ближайших соседей
        psi_temp[1:-1, 1:-1] = .25*(psi[2:, 1:-1] + psi[:-2, 1:-1] +
                                    psi[1:-1, 2:] + psi[1:-1, :-2])
        # Обновляйте проверку сходимости для каждой 50-й итерации
        if itnum % 50 == 0:
            if np.sum(np.abs(psi_temp - psi))/np.sum(psi) < tol:
                print("Итераций: ", itnum + 1)
                break
        psi = psi_temp.copy()
        
    if itnum == maxit - 1:
        print("Максимальное количество итераций (%i) достигнуто!"%(maxit))
        
    return psi

Давайте приведем один пример.

In [ ]:
n = 64
tol = 1e-5
psi = jacobi(n, maxit=10000, tol=1e-5)

Затем мы визуализируем результат для двух физических интерпретаций, упомянутых выше.

In [ ]:
# Необходимый пакет
import matplotlib.pyplot as plt
plt.style.use('bmh')
%matplotlib inline

def plot(psi, n):
    # Вычислит градиент. v=psi_yy, u=psi_xx
    v, u = np.gradient(psi)

    x = np.linspace(0, 1, n) # x-axis
    vmag = np.sqrt(u**2 + v**2)

    # Создать фигуру
    fig, axarr = plt.subplots(1, 2, figsize=(10, 6), dpi=300)

    # Оси 1: Электрический потенциал и поле
    im1 = axarr[0].contourf(x, x, psi, 20)
    axarr[0].streamplot(x, x, -u, -v, color="k")
    axarr[0].set_title("Electric potential and field")
    fig.colorbar(im1, orientation='horizontal', ax=axarr[0],
                  label=r"Electric potential, $V/V_0$")
    axarr[0].set_xlabel("$x/L$")
    axarr[0].set_ylabel("$y/L$")

    # Оси 2: Поле скорости и напряженность
    # Течение несжимаемой и невращающейся жидкости
    im2 = axarr[1].contourf(x, x, vmag/np.max(vmag), 20)
    axarr[1].streamplot(x, x, v, -u, color="k")
    axarr[1].set_title("Velocity field and strength")
    fig.colorbar(im2, orientation='horizontal', ax=axarr[1],
               label=r"Velocity magnitude, $v/v_0$")
    axarr[1].set_xlabel("$x/L$")
    axarr[1].set_ylabel("$y/L$")
    axarr[1].set_xlim([0, 1])
    axarr[1].set_ylim([0, 1])

In [ ]:
plot(psi, n)

#### Общий случай
Метод Якоби может использоваться для решения общего матричного уравнения $A\mathbf x = \mathbf b$ (при условии, что метод сходится). Пусть $D$ - диагональная часть $A$, $L$ - нижняя треугольная часть $A$, а $U$ - верхняя треугольная часть $A$. Тогда метод Якоби может быть записан как

\begin{equation}
D\mathbf x^{\,(k+1)} = -(L+U)\mathbf x^{\,(k)} + \mathbf b.
\end{equation}

В приведенном выше примере $D$ было просто единичной матрицей, а $\mathbf b$ был неоднородностью $\rho$ (с переиндексированием, $\rho_{i, j}\to \rho_{i+jN}$).

## Метод Гаусса-Зейделя
Метод Гаусса-Зейделя аналогичен методу Якоби, но среднее значение в каждой точке вычисляется с использованием "новейших" доступных данных. Если усреднение выполняется по оси $x$($i$ изменяется, $j$ фиксируется), то

\begin{equation}
\psi^{n+1}_{i, j} = \frac{1}{4} \left(\psi_{i+1, j}^n + \psi_{i-1, j}^{n+1} + \psi_{i, j+1}^n + \psi_{i, j-1}^{n+1}\right) - \frac{\Delta ^2}{4} \,\rho_{i, j}.
\label{eq:gauss-seidel_method}\quad (5)
\end{equation}

Метод Гаусса-Зейделя в матричном виде

\begin{equation}
(L+D)\mathbf x^{\,(k)} = -U\mathbf x^{\,(k-1)}+ \mathbf b.
\label{eq:gauss-seidel_matrix}\quad (6)
\end{equation}

## Метод релаксации (SOR)
Метод Гаусса-Зейделя (6) может быть переписан как

$$\mathbf x^{\,(k)} = \mathbf x^{\,(k-1)} - (L + D)^{-1}\mathbf \xi^{\,(r-1)},$$

где $\mathbf \xi^{\,(r-1)}=\left[(L+D+U)\mathbf x^{\,(r-1)}-\mathbf b\right]$. $\xi$ это просто остаточный вектор, который мы переопределяем, вводя параметр *overrelaxation* $\omega$. Таким образом, метод SOR может быть записан как

\begin{equation}
\mathbf x^{\,(k)} = \mathbf x^{\,(k-1)} - \omega (L + D)^{-1}\mathbf \xi^{\,(r-1)}.
\end{equation}

Можно доказать, что метод SOR сходится только для $0<\omega < 2$ и только $1<\omega < 2$ может дать более быструю сходимость, чем методы Гаусса-Зайделя и Якоби [[1]](#rsc1) (см. [Приложение](#appendix) для быстрого обзора скорости сходимости). Метод называется *underrelaxed*, если $0<\omega < 1$. Одним из недостатков этого метода является то, что оптимальное значение для $\omega$ в общем случае не известно аналитически. В этих случаях можно найти приблизительное значение эмпирически.

Давайте создадим функцию, которая выполняет те же вычисления, что и в приведенном выше примере, чтобы сравнить метод Якоби и SOR. Обратите внимание, что мы не можем использовать Numpy-массивы для оптимизации скорости вычислений для методов SOR и Гаусса-Зейделя. Таким образом, число итераций может быть меньше, но вычислительное время намного больше, чем у метода Якоби. Эта "проблема" решается, если мы используем скомпилированный язык, такой как fortran, C или C++ (см., Например, Наш учебник по [F2PY](https://www.numfys.net/howto/F2PY/) для получения дополнительной информации).

Мы можем использовать остаток в качестве проверки сходимости. Чтобы сравнить два метода, мы остановим итерации, если общее относительное изменение между двумя итерациями будет ниже заданного допуска, как мы делали раньше.

In [ ]:
def sor(n=32, omega=1, maxit=1000, tol=1e-5):
    """ Эта функция использует метод релаксации Successive over-relaxation (SOR) 
        для решения уравнения Пуассона для квадратной области 
        и граничных условий, как показано на рис.1.
    
    Параметры:
        n :     int. Количество шагов в направлении x и y
        omega:  float. Параметр перенапряжения (0 < omega < 2)
        maxit : int. Максимальное количество итераций.
        tol :   float. Допустимая погрешность.
                Если общее относительное изменение между двумя
                итерациями меньше, чем "tol", мы говорим, что метод сошелся.
    Возвращается:
    float array, size(n, n). Расчетное решение. 
    """
    
    psi = bc(n) # начальное предположение и граничные условия

    for itnum in range(maxit):
        total_xi = 0
        for i in range(1, n - 1):
            for j in range(1, n - 1):
                xi = psi[i + 1, j] + psi[i - 1, j] + psi[i, j + 1] + psi[i, j - 1] - 4*psi[i, j]
                total_xi += abs(xi)
                psi[i, j] += omega*xi/4
        if .25*total_xi/np.sum(psi) < tol:
            print("Итераций: ", itnum + 1)
            break

    if itnum == maxit - 1:
        print("Максимальное количество итераций (%i) достигнуто!"%(maxit))

    return psi

In [ ]:
n = 64
w = 1.91 # Аппроксимация оптимального значения после нескольких тестов
psi = sor(n, omega=w, maxit=1000, tol=5e-5)

In [ ]:
plot(psi, n)

В заключение следует отметить, что SOR требовал гораздо меньше итераций, чем метод Якоби. Обратите внимание, что критерий остановки в SOR на самом деле несколько строже, чем в методе Якоби, поскольку скорость сходимости выше.

## Ускорение Чебычева
Существует несколько простых методов улучшения алгоритма SOR. Заданное оптимальное значение для $\omega$ для асимптотического поведения не обязательно является хорошим выбором для начального значения. Существует умный рецепт для выбора значения $\omega$, которое изменяется для каждой итерации. Кроме того, вы можете заметить, что "нечетные" сетки зависят только от предыдущих значений "четных" сеток, и наоборот. SOR с этими оптимизациями называется *Ускорение Чебычева* [[1]](#rsc1).

## Resources and Further Readings

<a name="rsc1">[1]</a> William H. Press, Saul A. Teukolsky, William T. Vetterling, and Brian P. Flannery. *Numerical Recipes 3rd Edition: The Art of Scientific Computing*. Cambridge University Press, 3 edition, September 2007.  
<a name="rsc2">[2]</a> David J. Griffiths. *Introduction to Electrodynamics* (3rd Edition). Benjamin Cummings, 1998.  
<a name="rsc3">[3]</a> F. Durst. *Fluid Mechanics an Introduction to the Theory of Fluid Flows*. Springer-Verlag Berlin Heidelberg. 2008. https://doi.org/10.1007/978-3-540-71343-2  

___

<a name="appendix"></a>
## Appendix
#### Теорема единственности для уравнения Пуассона
Рассмотрим уравнение Пуассона

$$\nabla^2\psi(\mathbf r) = -\rho(\mathbf r)_f,$$

для некоторого реального потенциала $\psi$ и источника $\rho$. Предположим, что существуют два различных решения $\psi_1(\mathbf r)$ и $\psi_2(\mathbf r)$. Взяв разность уравнений Пуассона для каждого из решений, мы получим

$$0 = \nabla^2\psi_1(\mathbf r) - \nabla^2\psi_2(\mathbf r) = \nabla^2\left[\psi_1(\mathbf r) - \psi_2(\mathbf r)\right].$$

Определим $\phi(\mathbf r)\equiv \psi_1(\mathbf r) - \psi_2(\mathbf r)$. Обратите внимание, что

$$\nabla (\phi\nabla\phi)=(\nabla\phi)^2+ \phi\nabla^2\phi=(\nabla\phi)^2.$$

Теперь мы возьмем интеграл по некоторому объему $V$, ограниченному поверхностью $S$ с обеих сторон:

$$\int_V\mathrm{d}^3\mathbf{r}\;(\nabla\phi)^2=\int_V\mathrm{d}^3\mathbf{r}\;\nabla(\phi\nabla\phi).$$

Последний интеграл может быть переписан как поверхностный интеграл по $S$ с помощью теоремы *дивергенции*. Таким образом, мы получаем

$$\int_S \phi\nabla\phi\cdot \mathrm d\mathbf a = \int_V\mathrm{d}^3\mathbf{r}\;(\nabla\phi)^2.$$

Обратите внимание, что $(\nabla \phi)^2\geq 0$ везде, что означает, что $\nabla \phi=0$, если

$$\int_S \phi\nabla\phi\cdot \mathrm d\mathbf a=0.$$

Другими словами, *градиент* решения уникален, когда интеграл поверхности обращается в нуль. Это тот случай, когда на границах указывается $\phi$ или $\nabla \phi$. Это называется граничными условиями Дирихле и Неймана соответственно. Граничные условия также могут быть описаны в $r\to \infty$, если поверхностный интеграл обращается в нуль.

Теорема единственности уравнения Лапласа для электромагнетизма обсуждается в ref. [[2]](#rsc2).

#### Анализ устойчивости Фон Неймана для разностной схемы FTCS уравнения Пуассона

Ниже приводится краткое применение анализа устойчивости Фон Неймана для конечного дифференцирования FTCS (Прямое время, центрированное пространство). Короче говоря, он состоит из линеаризации уравнения и разделения пространственных и временных частей, а также рассмотрения роста локальной ошибки. Для получения дополнительной информации и более глубокого понимания см. например [chap. 20, [1](#rsc1)].

Предположим, что решения разностного уравнения можно записать в виде

\begin{equation}
\psi^n_{j, l}=\xi^n(k, q)e^{ikj\Delta x}e^{iql\Delta y},
\label{eq:fourier}\quad (7)
\end{equation}

где $i$ - мнимая единица, а $k$ и $q$ - действительные числа. *Фактор роста* $G=\xi^{n+1}/\xi^n$ схемы определяется путем подстановки приведенного выше выражения в разностное уравнение. Метод стабилен, если $|G|\leq 1$ для всех реальных $k$, и нестабилен, если $|G|>1$ для некоторых реальных $k$.

В нашем случае мы должны вставить (7) в (3) и использовать $\Delta \equiv\Delta x = \Delta y$. В областях без источника ($\rho = 0$):

$$
\begin{aligned}
G &= 1 + 2\frac{\Delta t}{\Delta^2}\left(\cos(k\Delta) + \cos(q\Delta) - 2\right)\\
&=1 - 4\frac{\Delta t}{\Delta^2}\left(\sin^2(k\Delta/2) + \sin^2(q\Delta/2)\right).
\end{aligned}
$$

Если мы потребуем, чтобы $|G|\leq 1$ для всех $k$ и $q$, мы получим критерий устойчивости

\begin{equation}
\frac{\Delta t}{\Delta^2}\leq \frac{1}{4}.
\end{equation}

#### Скорость сходимости
Методы Якоби и Гаусса-Зейделя медленно сходятся. Можно показать, что количество итераций $r$, необходимых для уменьшения ошибки в 10 раз ${- p}$, оценивается

\begin{align*}
r&\approx \frac{1}{2}pN^2\quad\text{(Jabobi)}, \\
r&\approx \frac{1}{4}pN^2\quad\text{(Gauss-Seidel)},
\end{align*}

где $N^2$ - количество точек сетки [[1]](#rsc1). Метод SOR в целом намного быстрее, чем методы Якоби и Гаусса-Зейделя для оптимального параметра $\omega$, для которого требуется только порядка $N$ итераций. Можно показать, что оптимальное значение для $\omega$ при больших $N$ может быть оценено

$$\omega_\text{opt}\approx\frac{2}{1+\pi/N},$$

и что

\begin{equation*}
r\approx \frac{1}{3}pN\quad\text{(SOR)}.
\end{equation*}